## Imports



In [ ]:
from common_language import _LANGUAGES
import processing as prlib

from scipy.fft import fft
from mutagen.wave import WAVE
from parselmouth.praat import call
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import librosa
import librosa.display
import parselmouth
import noisereduce as nr
import os 



##Features processing

###Get data frames

In [ ]:
print(os.getcwd())
_, train_df, test_df, validation_df = prlib.get_clean_dataframes()
train_df.head(), prlib.frame_length

###MFCCs

###Spectral measurements

###Pitch sequence

###Formants

###RMS and ZCR

###HNR

### Features matrix

In [ ]:
training_audios = train_df['paths'].tolist()
test_audios = test_df['paths'].tolist()
validation_audios = validation_df['paths'].tolist()

def extract_zcr_features(zcr_vector, hnr_mean): 
    # Calculate aggregated features
    features = {
        "mean_zcr": np.mean(zcr_vector),
        "std_dev_zcr": np.std(zcr_vector),
        "median_zcr": np.median(zcr_vector),
        "min_zcr": np.min(zcr_vector),
        "max_zcr": np.max(zcr_vector),
        "q25_zcr": np.percentile(zcr_vector, 25),
        "q75_zcr": np.percentile(zcr_vector, 75),
        "iqr_zcr": np.percentile(zcr_vector, 75) - np.percentile(zcr_vector, 25),
        "hnr_mean": hnr_mean
    }

    # Convert the features dictionary to a pandas DataFrame
    features_df = pd.DataFrame([features])
    
    return features_df    

def extract_rms_features(rms_energy): 
    features = {
            "mean_energy": np.mean(rms_energy),
            "std_dev_energy": np.std(rms_energy),
            "median_energy": np.median(rms_energy),
            "min_energy": np.min(rms_energy),
            "max_energy": np.max(rms_energy),
            "energy_range": np.max(rms_energy) - np.min(rms_energy),
            "q25_energy": np.percentile(rms_energy, 25),
            "q75_energy": np.percentile(rms_energy, 75),
            "energy_iqr": np.percentile(rms_energy, 75) - np.percentile(rms_energy, 25),
            "energy_variability": np.sum(np.abs(np.diff(rms_energy))),
            "zero_crossing_rate": np.sum(np.diff(np.sign(np.diff(rms_energy))) != 0) / len(rms_energy),
            "low_energy_frame_rate": np.sum(rms_energy < (0.5 * np.mean(rms_energy))) / len(rms_energy)
        }

    features_df = pd.DataFrame([features])

    return features_df


def extract_formants(f1, f2, f3): 
    f1, f2, f3 = map(lambda x: np.nan_to_num(np.asarray(x)), [f1, f2, f3])
    features = {}
    for formant, name in zip([f1, f2, f3], ['F1', 'F2', 'F3']):
        features[f'{name}_mean'] = np.nanmean(formant)
        features[f'{name}_std_dev'] = np.nanstd(formant)
        features[f'{name}_median'] = np.nanmedian(formant)
        features[f'{name}_min'] = np.nanmin(formant)
        features[f'{name}_max'] = np.nanmax(formant)
        features[f'{name}_range'] = np.nanmax(formant) - np.nanmin(formant)
        features[f'{name}_q25'] = np.nanpercentile(formant, 25)
        features[f'{name}_q75'] = np.nanpercentile(formant, 75)
        features[f'{name}_iqr'] = np.nanpercentile(formant, 75) - np.nanpercentile(formant, 25)
        # Ensure there are at least 2 elements to compute diff, otherwise default to 0
        features[f'{name}_delta_sum'] = np.sum(np.abs(np.diff(formant))) if len(formant) > 1 else 0

    if all(len(formant) > 0 for formant in [f1, f2, f3]):
        features['F2_F1_diff_mean'] = np.nanmean(f2 - f1)
        features['F3_F2_diff_mean'] = np.nanmean(f3 - f2)
    else:
        features['F2_F1_diff_mean'], features['F3_F2_diff_mean'] = 0, 0

    features_df = pd.DataFrame([features])
    return features_df

def extract_spectre(data, type):
    # Define feature names
    feature_names = ['mean_' + type, 'std_' + type, 'median_' + type, 'min_' + type, 'max_' + type, 'q25_' + type, 'q75_' + type]
    
    # Compute the features
    mean_val = np.mean(data)
    std_val = np.std(data)
    median_val = np.median(data)
    min_val = np.min(data)
    max_val = np.max(data)
    q25, q75 = np.percentile(data, [25, 75])
    
    # Collect features into a list
    features = [mean_val, std_val, median_val, min_val, max_val, q25, q75]
    
    # Create a DataFrame from the features list
    features_df = pd.DataFrame([features], columns=feature_names)
    
    return features_df

# Outputs 3 features per band (Contrast peak, temporal evolution, rate of change) = 21 
# Mean and std of spectral contrast
def extract_contrast(spectral_contrast):
    features = []
    feature_names = []

    # Iterate over each frequency band to calculate band-specific features
    for band in range(spectral_contrast.shape[0]):
        contrast_band = spectral_contrast[band, :]

        # Count significant peaks
        peaks, _ = find_peaks(contrast_band, height=np.mean(contrast_band))
        features.append(len(peaks))
        feature_names.append(f'band_{band}_peaks')

        # Temporal evolution: difference between means of the first and second halves
        mid_point = len(contrast_band) // 2
        mean_diff = np.mean(contrast_band[mid_point:]) - np.mean(contrast_band[:mid_point])
        features.append(mean_diff)
        feature_names.append(f'band_{band}_mean_diff')

        # Rate of change (derivative)
        derivative = np.mean(np.abs(np.diff(contrast_band)))
        features.append(derivative)
        feature_names.append(f'band_{band}_derivative')

    # Add overall statistical measures for the entire spectral contrast matrix
    overall_mean = np.mean(spectral_contrast)
    features.append(overall_mean)
    feature_names.append('overall_mean')

    overall_std = np.std(spectral_contrast)
    features.append(overall_std)
    feature_names.append('overall_std')

    # Convert the features list into a DataFrame
    features_df = pd.DataFrame([features], columns=feature_names)

    return features_df

def mfccs_to_df(mfcc_means,mfcc_stds): 
    # Ensure mfcc_means and mfcc_stds are flat arrays
    mfcc_means = mfcc_means.flatten()
    mfcc_stds = mfcc_stds.flatten()
    
    # Generate column names
    mean_col_names = [f'MFCC_mean_{i+1}' for i in range(len(mfcc_means))]
    std_col_names = [f'MFCC_std_{i+1}' for i in range(len(mfcc_stds))]
    
    # Combine the MFCC means and stds into a single DataFrame
    mfcc_features_df = pd.DataFrame([np.concatenate([mfcc_means, mfcc_stds])],
                                    columns=mean_col_names + std_col_names)
    
    return mfcc_features_df

type = [(training_audios, 'train'), (test_audios, 'test'), (validation_audios, 'validation')]
total = len(training_audios) + len(test_audios) + len(validation_audios)
c=0
for i in range(len(type)):
    attributes_df = pd.DataFrame()
    for j in range(300, len(type[i][0])):
        try:
            print(c, "/", total, ":", type[i][0][j])
            c+=1
            path = prlib.get_clean_path(type[i][1], type[i][0][j])
            print(type[i][0][j] + " " + path)
            #sample_rate represent the number of samples per seconds in the original signal.
            data = prlib.get_data(path)
            #clean data

            #Get the attributes
            mfccs = prlib.get_Normalized_Mfccs(data)
            specs_measurements = prlib.get_spectral_measurements(data)
            pitch_track = prlib.get_pitch_sequences(data)
            formants_data = prlib.get_formants(path)
            rms_energy = prlib.get_rms_energy(data)
            zcr = prlib.get_ZCR(data)
            hnr_mean = prlib.get_HNR(data)
            # mfccs: Get mean and std atributes
            mfccs = mfccs_to_df(np.mean(mfccs, axis = 1),np.std(mfccs, axis = 1))

            # Spec measurements: Use extract specter to produce features
            spectre_centroid_df = extract_spectre(specs_measurements[0], "centroid")
            spectre_rollof_df = extract_spectre(specs_measurements[1], "rollof")
            spectre_bandwidth_df = extract_spectre(specs_measurements[2], "bandwidth")
            spectre_flatness_df = extract_spectre(specs_measurements[3], "flatness")
            

            # Spectre contrast
            spectre_contrast_df = extract_contrast(specs_measurements[4])

            # Pitch track: use extract spectre method
            # Add IQR and pitch delta sum -> 9 features
            #print(pitch_track.shape)
            pitch_track_df = extract_spectre(pitch_track, "pitch_track")
            # Formants : Call extract_formants with f1, f2, f3
            # Returns dataframe with 32 features
            formants_df = extract_formants(formants_data["F1"], formants_data["F2"], formants_data["F3"])
            
            # RMS
            rms_energy_df = extract_rms_features(rms_energy[0])

            if type[i][1] == 'train':
                label = pd.Series(train_df['language'][j])
            elif type[i][1] == 'test':
                label = pd.Series(test_df['language'][j])
            elif type[i][1] == 'validation':
                label = pd.Series(validation_df['language'][j])

            #ZCR and HNR mean
            combined_features_row = pd.concat([mfccs, spectre_centroid_df,\
                                            spectre_rollof_df, \
                                                spectre_bandwidth_df,\
                                                      spectre_flatness_df, \
                                                        spectre_contrast_df,\
                                                            pitch_track_df,\
                                                                formants_df, \
                                                                    rms_energy_df, label], axis = 1)

            """
            row = pd.DataFrame({'Audio': audio ,'MFCCs': [np.array(mfccs)],
                                        'Spec Centroid': [specs_measurements[0]], 'Spec Rollof': [specs_measurements[1]],
                                        'Spec Bandwidth': [specs_measurements[2]], 'Spec Flatness': [specs_measurements[3]],
                                        'Spec Contrast': [specs_measurements[4]], 'Pitch Track': [pitch_track],
                                        'Formants': [formants_data], 'RMS Energy': [rms_energy],
                                        'ZCR': [zcr], 'HNR Mean': [hnr_mean]})
            """
            attributes_df = pd.concat([attributes_df, combined_features_row], ignore_index=True)
        except Exception as e:
            print(e)
    """
    if type[i][1] == 'train':
        attributes_df['label'] = train_df['language']
    elif type[i][1] == 'test':
        attributes_df['label'] = test_df['language']
    elif type[i][1] == 'validation':
        attributes_df['label'] = validation_df['language']
      
    attributes_df.to_csv(type[i][1] + "_preprocessed_data")
    """

In [ ]:
attributes_df

## Features visualisation

### Initialisation

In [ ]:
data, sample_rate = librosa.load(path=f'./wav_files/validation/{validation_df['paths'][0]}')
data = nr.reduce_noise(y=data, sr=sample_rate)

### MFCCs visualisation

In [ ]:
normalized_mfccs = get_Normalized_Mfccs(data, sample_rate)

plt.figure(figsize=(12,6))
librosa.display.specshow(normalized_mfccs, x_axis='time')
plt.colorbar()
plt.title('MFCCs')
plt.tight_layout()
plt.show()

### Spectral measurements visualisation

In [ ]:
specs = get_spectral_measurements(data, sample_rate)

t = librosa.frames_to_time(range(len(specs[0])), sr=sample_rate)

# Plotting the Spectral Features
plt.figure(figsize=(12, 4))
plt.plot(t, specs[0], color='red', label='Centroid')
plt.plot(t, specs[1], color='blue', label='Rolloff')
plt.plot(t, specs[2], color='green', label='Bandwidth')

# For spectral flatness, there's no need to convert to dB since it's a ratio and typically small.
#plt.plot(t, spectral_flatness, color='orange', label='Flatness')

# For spectral contrast, it's common to average over the frequency bands since it returns an array of shape (n_bands, n_frames).
#spectral_contrast_avg = np.mean(spectral_contrast, axis=0)
#plt.plot(t, spectral_contrast_avg, color='black', label='Contrast')

plt.xlabel("Time (s)")
plt.ylabel("Spectral Feature Value")
plt.title("Spectral Features Over Time")
plt.legend(loc='best')
plt.show()

### Pitches visualisation

In [ ]:
pitch_track = get_pitch_sequences(data, sample_rate)
plt.figure(figsize=(12, 6))
plt.plot(pitch_track)
plt.xlabel('Time (frames)')
plt.ylabel('Frequency (Hz)')
plt.title('Pitch Track')
plt.show()

### RMS Visualisation

In [ ]:
rms_energy = get_rms_energy(data)
frames = range(len(rms_energy[0]))
t = librosa.frames_to_time(frames, sr=sample_rate)

plt.figure(figsize=(12, 4))
plt.plot(t, rms_energy[0], label='RMS Energy')
plt.xlabel("Time (s)")
plt.ylabel("Energy")
plt.title("RMS Energy Over Time")
plt.legend()
plt.show()